In [82]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db


# Fetch the service account key JSON file contents
cred = credentials.Certificate('/Users/Solomon/Desktop/cau-hashkeyword-serviceAccountKey.json')

# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://cau-hashkeyword.firebaseio.com'
})

ValueError: Invalid refresh token configuration. JSON must contain a "type" field set to "authorized_user".

In [73]:
class Info_webpage:
    def setData(self, title, date, url):
        self.title = title
        self.date = date
        self.url = url
    def getData(self):
        self.dataSet = []
        for i in range(len(self.title)):
            self.dataSet.append((self.title[i], self.date[i], self.url[i]))
        return self.dataSet
    def getTitle(self):
        return self.title
    def getDate(self):
        return self.date
    def getUrl(self):
        return self.url

In [74]:
class Info_user:
    def __init__(self):
        self.match = []
    def setData(self, user, keywords, selectedWebsite, fcmToken): # 나중에 uid 부분은 빼도 무방
        self.uid = user
        self.keywords = keywords
        self.selectedWebsite = selectedWebsite
        self.fcmToken = fcmToken
    def setSelectedWebsite(self, selectedWebsite): # parsing한 웹페이지 이름 저장
        self.selectedWebsite = selectedWebsite
    def getUid(self):
        return self.uid
    def getKeywords(self):
        return self.keywords
    def getSelectedWebsite(self):
        return self.selectedWebsite
    def getFcmToken(self):
        return self.fcmToken
    def setPushData(self, matchKeyword, matchTitle, matchDate, matchURL):
        self.match.append((matchKeyword, matchTitle, matchDate, matchURL))
    def getPushData(self):
        return self.match
    def initPushData(self): # Push 알림을 보낸 후 초기화 과정
        self.match = []

In [75]:
ref = db.reference('crawling/webpages')

dorm = Info_webpage()
ict = Info_webpage()

webpages = {'dormitory': dorm, 'ict': ict}

for (name, page) in webpages.items():
    page.setData(ref.child(name).child('title').get(),
                 ref.child(name).child('date').get(),
                 ref.child(name).child('url').get())

In [76]:
ref = db.reference('users')

users = ref.get() # 유저의 uid 가져옴
# users_uid = ref.get().keys()
users_info = []

for user in users:
    infoUser = Info_user() # 객체를 유저 하나씩 만듬
    infoUser.setData(user,
                     ref.child(user).child('keywords').get(),
                     ref.child(user).child('selectedWebsite').get(),
                     ref.child(user).child('fcmToken').get())
    user_parseSelectedWebsite = []
    for selected in infoUser.getSelectedWebsite():
        user_parseSelectedWebsite.append(selected.split('(')[1][:-1].split('.')[0])
    infoUser.setSelectedWebsite(user_parseSelectedWebsite)
    users_info.append(infoUser)

In [77]:
# 4중 for문 최적화 요망 (추후 업데이트) - 물론 크롤링 주기를 짧게하면 title을 도는 부분이 적게 걸림
# 유저를 순차적으로 선택하며 해당 유저가 선택한 웹사이트에서 키워드를 공지사항의 제목과 비교

for user in users_info:
    for site in user.getSelectedWebsite():
        for keyword in user.getKeywords():
            for (title, date, url) in webpages[site].getData():
                if keyword in title:
                    user.setPushData(keyword, title, date, url)

In [69]:
# ios Application에 push notification 전송

In [70]:
# user별로 push할 데이터를 담아둔 match 리스트 초기화 (match는 이번 실행에 보낼 데이터만을 저장해 둠)
# for user in users_info:
#    user.initPushData()

In [78]:
fcmTokens = {}

for user in users_info:
    if(len(user.getPushData()) > 0):
        print(user.getUid())
        print(user.getFcmToken())
        print(user.getPushData())
        print()
        fcmTokens[user.getFcmToken()] = user.getPushData()
        
fcmTokens # 현재는 같은 기기에 아이디를 2개 등록하였기 때문에 fcmToken이 덮어 쓰여졌다.

hw2HvzaMONf1IPjWsC0kDa7a1Ta2
cRxLJsIeJQo:APA91bGigr4N6dy8DSc-_g40fPst6wV2mBEcmLX0sNdk4y5HnWgoalpo35KMQL1GUOS9GAnnP6oHAG4ucX2yNxFOphaZHylFtCvzeiw9P1-kl7VVqgAuNKfNo86rt3Z90UyhVDHt2MMy
[('막역지룸', '[막역지룸] 2019학년도 1학기 "막역지룸(莫逆之Room)" 행사 안내/“Giving Compliments to Your Roommate” Program Information', '2019.04.03', 'https://dormitory.cau.ac.kr/bbs/bbs_view.php?pNum=2769&bbsID=notice'), ('막역지룸', '[막역지룸] 2019-1학기 막역지룸 선발 공지/Roommate program notice', '2019.04.17', 'https://dormitory.cau.ac.kr/bbs/bbs_view.php?pNum=2780&bbsID=notice')]

sVEZQkUcIsg5JmPDgrt3sQWu6yq2
cRxLJsIeJQo:APA91bGigr4N6dy8DSc-_g40fPst6wV2mBEcmLX0sNdk4y5HnWgoalpo35KMQL1GUOS9GAnnP6oHAG4ucX2yNxFOphaZHylFtCvzeiw9P1-kl7VVqgAuNKfNo86rt3Z90UyhVDHt2MMy
[('입관', '[생활관] 생활관 입관 가능시간 안내/Dormitory Admission Time Information', '2016.06.23', 'https://dormitory.cau.ac.kr/bbs/bbs_view.php?pNum=1611&bbsID=notice')]



{'cRxLJsIeJQo:APA91bGigr4N6dy8DSc-_g40fPst6wV2mBEcmLX0sNdk4y5HnWgoalpo35KMQL1GUOS9GAnnP6oHAG4ucX2yNxFOphaZHylFtCvzeiw9P1-kl7VVqgAuNKfNo86rt3Z90UyhVDHt2MMy': [('입관',
   '[생활관] 생활관 입관 가능시간 안내/Dormitory Admission Time Information',
   '2016.06.23',
   'https://dormitory.cau.ac.kr/bbs/bbs_view.php?pNum=1611&bbsID=notice')]}

In [83]:
# Send to single device.
from pyfcm import FCMNotification
import json

with open("/Users/Solomon/Desktop/fcm-server-key.json") as json_file:
    json_data = json.load(json_file)
    server_key = json_data["server_key"]

push_service = FCMNotification(api_key=server_key)

# OR initialize with proxies
# proxy_dict = {
#           "http"  : "http://127.0.0.1",
#           "https" : "http://127.0.0.1",
#         }
# push_service = FCMNotification(api_key="<api-key>", proxy_dict=proxy_dict)

# Your api-key can be gotten from:  https://console.firebase.google.com/project/<project-name>/settings/cloudmessaging

i = 0
for (token, data) in fcmTokens.items():
    registration_id = token
    message_title = "중앙대학교(서울) " + "생활관 " + "#" + data[i][0] # 생활관 같은 공지사항의 위치는 추후 다시 처리하도록 하자
    message_body = data[i][1]
    result = push_service.notify_single_device(registration_id=registration_id, message_title=message_title, message_body=message_body)
    i += 1

    print(result)

{'multicast_ids': [4920976223142322881], 'success': 1, 'failure': 0, 'canonical_ids': 0, 'results': [{'message_id': '1561357783471988'}], 'topic_message_id': None}
